In [21]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

通过使用深度学习框架来简洁地实现线性回归模型 生成数据集

In [22]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有地API来读取数据

In [23]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  #将之前得到的特征和标签通过data.TensorDatasetd写入dataset里生成一个数据集，*表示将列表中的元素当作参数写入
    return data.DataLoader(dataset, batch_size, shuffle=is_train)   #通过data.DataLoader将dataset数据集生成多个大小为batch_size的小批量样本

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [24]:
next(iter(data_iter))

[tensor([[-0.2144, -0.1956],
         [ 1.1507, -0.6552],
         [ 1.2677,  0.4222],
         [-1.5072, -0.5720],
         [-2.2534,  0.6889],
         [-0.4819,  0.3158],
         [-0.0175, -0.1036],
         [-0.8153, -0.3864],
         [ 1.6017, -2.0646],
         [-2.1019,  1.7204]]),
 tensor([[ 4.4285],
         [ 8.7357],
         [ 5.2851],
         [ 3.1336],
         [-2.6369],
         [ 2.1612],
         [ 4.5185],
         [ 3.8758],
         [14.4295],
         [-5.8590]])]

In [25]:
# 建立线性回归神经网络模型，nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))    #指定训练模型为线性回归网络，且输入维度2，输出维度1  sequential=ist of layers,相当于网络层级的一个容器

In [26]:
#初始化模型参数
net[0].weight.data.normal_(0, 0.01)     #初始化w，以均值为0，方差为0.01的正态分布
net[0].bias.data.fill_(0)       #初始化b=0

tensor([0.])

In [27]:
#定义损失函数为平方范数
loss = nn.MSELoss()

In [28]:
#实例化sgd实例  即优化算法  其中学习率设置为0.03
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [29]:
#训练过程 迭代次数为三次
num_epochs = 8
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}') #{l : f} -> 打印l，格式为浮点型

epoch 1, loss 0.000239
epoch 2, loss 0.000103
epoch 3, loss 0.000102
epoch 4, loss 0.000103
epoch 5, loss 0.000103
epoch 6, loss 0.000102
epoch 7, loss 0.000103
epoch 8, loss 0.000103


In [30]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0002, 0.0007])
b的估计误差： tensor([0.0002])
